In [234]:
import os
import copy
os.getcwd()
import numpy as np

In [5]:
import sys
sys.path.append('/Users/aveek/development/chiko-deals/deals-getter/utils')

In [135]:
import requests
import json

def countPostalCodesFromRawAddress(fullAddress):
    postalCodes = []
    for i in range(len(fullAddress)):
        if fullAddress[i] == ')' and i - len("Singapore (123456") >= 0:
            if fullAddress[i - len("Singapore (123456"): i - len("Singapore (123456") + len("Singapore (")] == "Singapore (":
                postalCodes.append(fullAddress[i - len("123456"): i])
    return len(postalCodes)

        

def getLongLatFromRawAddress(fullAddress):
    postalCodes = []

    for i in range(len(fullAddress)):
       if fullAddress[i] == ')' and i - len("Singapore (123456") >= 0:
            if fullAddress[i - len("Singapore (123456"): i - len("Singapore (123456") + len("Singapore (")] == "Singapore (":
                postalCodes.append(fullAddress[i - len("123456"): i])
        
    longLats = []
    for postal in postalCodes:
        longLats.append(getLongLatFromPostal(postal))
    return longLats
                

def getLongLatFromPostal(postal):
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={postal}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    headers = {"Authorization": "**********************"}
    response = requests.request("GET", url, headers=headers)
    # Parse the response text into a dictionary
    data = json.loads(response.text)

    if data['found'] == 0:
        print("Invalid postal given")
        return None
        
    # Access the latitude and longitude from the dictionary
    latitude = data['results'][0]['LATITUDE']
    longitude = data['results'][0]['LONGITUDE']
    building_name = data['results'][0]['BUILDING']
    print(f"Queried long lat for: {building_name}")

    return longitude, latitude

if __name__ == "__main__":
    pass

In [136]:
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
# from utils.chopeDetails import parseTags
# from utils.addressConverter import getLongLatFromRawAddress, countPostalCodesFromRawAddress

def get_webscrape_data():
    ## selenium used to scroll to the bottom of the page
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    driver.get("https://shop.chope.co/collections/best-sellers")

    SCROLL_PAUSE_TIME = 0.5

    # Get the height of the page
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new height and check if we have reached the end of the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        
        last_height = new_height

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # getting the relevant data
    all_deals = soup.find(id = 'bc-sf-filter-products')
    url = 'https://shop.chope.co'
    promos = []
    longlat_list = []
    
    for deal in all_deals:
        title = getPromoTitle(deal)
        link = getPromoLink(deal)
        
        deal_link = url+link
        deal_page = requests.get(deal_link)
        deal_soup = BeautifulSoup(deal_page.content, 'html.parser')
        info = getPromoInfo(deal_soup)
        
        cur_deal_vouchers = getPromoVouchers(deal_soup)

        addressesAndOpeningHours = getPromoAddressAndOpeningHours(deal_soup)

        tagsList = getPromoTags(deal_soup)        
        image_url = getPromoImageUrl(deal)

        appendPromoGivenData(promos, title, info, addressesAndOpeningHours, image_url, tagsList, cur_deal_vouchers, deal_link)
        
    # maybe offset the latlong coords here
        
        
        
    return promos

def getPromoTitle(deal):
    return deal.select('a.color-blue.app-link')[0].text # restaurant / name of deal

def getPromoLink(deal):
    links = deal.select('a')
    link = links[0].get('href') # get url to go to the deal page
    return link

def getPromoInfo(deal_soup):
    infos = deal_soup.select('div.product-desc')
    info = infos[0].select('strong')[0].text #brief info on deal
    return info

def getPromoVouchers(deal_soup):
    # Get voucher details
    vouchers_div = deal_soup.select('div.product-variants.hide-mobile li.child.relative')
    cur_deal_vouchers = []
    for voucher in vouchers_div:
        # Extracting date
        date = voucher.select_one('div.date.color-blue.body-s.bold-700')
        if date:
            date = date.text.strip()

        # Extracting time
        time_ = voucher.select_one('div.time.body-s.color-darkgrey.mb-5')
        if time_:
            time_ = time_.text.strip()

        # Extracting price (both discounted and original)
        price_discounted = voucher.select_one('div.product-price strong.price.color-orange')
        if price_discounted:
            price_discounted = price_discounted.text.strip()
        
        price_original = voucher.select_one('div.product-price strike.color-darkgrey.body-s')
        if price_original:
            price_original = price_original.text.strip()

        product_savings = voucher.select_one('div.child-right div.product-savings.big')
        if product_savings:
            product_savings = product_savings.text.strip()

        cur_voucher = {"date": date, "time": time_, "price_discounted": price_discounted, "price_original": price_original, "product_savings": product_savings}
        cur_deal_vouchers.append(cur_voucher)
    return cur_deal_vouchers

def getPromoImageUrl(deal):
    image_tag = deal.select_one('.product-each-tile-image img')
    image_url = None
    if image_tag:
        image_url = image_tag['src']
    return image_url

def getPromoTags(deal_soup):
    cards = deal_soup.select('div.details')
    for card in cards:
        header = card.select('h5.header-xs.color-blue')
        
        if header == []:
            continue
        
        if header[0].text == "Cuisine":
            cuisine_div = card.select_one('div.rte')
            if cuisine_div:
                tags_unparsed = cuisine_div.text
                tagsList = parseTags(tags_unparsed)
                return tagsList

def getPromoAddressAndOpeningHours(deal_soup):
    """Returns a list of lists of addresses and opening hours for each outlet of the deal
    Returns:
        list: list of [address, opening_hours] for each outlet of the deal, where address is a string and opening_hours is a string
    """
    # Deal has multiple outlets
    if checkIfPromoHasMultipleOutlets(deal_soup):
        return getMultiOutletAddressAndOpeningHours(deal_soup)
    # Deal has one outlet/ address
    else:
        return getSingleOutletAddressAndOpeningHours(deal_soup)

def getMultiOutletAddressAndOpeningHours(deal_soup):
    concatenatedAddressAndOpeningHours = ""
    cards = deal_soup.select('div.details')
    for card in cards:
        header = card.select('h5.header-xs.color-blue')
        if header == []:
            continue
        if header[0].text == "Address":
            address_div = card.select_one('div.rte')
            if address_div:
                for br in address_div.find_all("br"):
                    br.replace_with(" ")  # Replace <br> with a space

                for p_tag in address_div.select('p'):
                    # Keep the <strong> tags by getting the HTML content of p_tag
                    p_html_content = p_tag.encode_contents().decode('utf-8')
                    concatenatedAddressAndOpeningHours += p_html_content + " "
    return parse_addresses_and_hours(concatenatedAddressAndOpeningHours)

# def parse_addresses_and_hours(input_str):
#     # Split the input string based on <strong> tags
#     parts = re.split(r'<strong>[^<]+</strong>', input_str)
#     parts = [part.strip() for part in parts if part.strip()]  # Remove empty or whitespace-only strings
    
#     results = []
    
#     for part in parts:
#         # Find the postal code (format: Singapore (******))
#         postal_code_match = re.search(r'Singapore \(\d{6}\)', part)
#         if postal_code_match:
#             postal_code_end = postal_code_match.end()
            
#             # Extract the address and opening hours
#             address = part[:postal_code_end].strip()
#             opening_hours = part[postal_code_end:].strip()
            
#             results.append([address, opening_hours])
    
#     return results

def parse_addresses_and_hours(input_str):
    if "<strong>" not in input_str:
        # Regular expression pattern to match each address and its opening hours
        pattern = r'([a-zA-Z0-9\s\-\#\,]+Singapore \(\d{6}\)) (Mon(?:-[^:]+)?: [^ ]+(?: [^ ]+)?(?:, )?)*(?:Sat(?:-[^:]+)?: [^ ]+(?: [^ ]+)?(?:, )?)*(?:Sun(?:-[^:]+)?: [^ ]+(?: [^ ]+)?(?:, )?)?'
        
        # Find all matches using the regular expression pattern
        matches = re.findall(pattern, input_str)
        
        # Initialize an empty list to store the results
        results = []
        
        # Loop through each match to extract the address and opening hours
        for match in matches:
            address, opening_hours = match
            results.append((address.strip(), opening_hours.strip()))
        
        return results
    
    # Split the input string based on <strong> tags, but keep the text within the tags
    parts = re.split(r'(<strong>[^<]+</strong>)', input_str)
    parts = [part.strip() for part in parts if part.strip()]  # Remove empty or whitespace-only strings
    
    results = []
    
    for i in range(0, len(parts), 2):
        strong_text = parts[i]  # Text within <strong> tags
        part = parts[i + 1]  # The address and opening hours part
        
        # Find the postal code (format: Singapore (******))
        postal_code_match = re.search(r'Singapore \(\d{6}\)', part)
        if postal_code_match:
            postal_code_end = postal_code_match.end()
            
            # Extract the address and opening hours
            address = strong_text + " " + part[:postal_code_end].strip()
            opening_hours = part[postal_code_end:].strip()
            
            results.append([address, opening_hours])
    
    return results

def getSingleOutletAddressAndOpeningHours(deal_soup):
    cards = deal_soup.select('div.details')
    address = None
    opening_hours = None
    for card in cards:
        header = card.select('h5.header-xs.color-blue')
        
        if header == []:
            continue

        if header[0].text == "Address":
            address = str(card.select('p')[0])
            address = strip_things(address)  # address of restaurant
        
        if header[0].text == "Opening Hours":
            opening_hours_div = card.select_one('div.rte')
            if opening_hours_div:
                opening_hours = opening_hours_div.text.replace("\n", "")
    if opening_hours == None:
        # There are 2 ways, from the given scraped data. If there are more new ways to format then these 2 way will not suffice
        # if parse_addresses_and_hours(address)[0][1] != '':
        #     return parse_addresses_and_hours(address)
        address_div = card.select_one('div.rte')
        if address_div:
            for br in address_div.find_all("br"):
                br.replace_with(" ")  # Replace <br> with a space
            concatenatedAddressAndOpeningHours = ""
            for p_tag in address_div.select('p'):
                # Keep the <strong> tags by getting the HTML content of p_tag
                p_html_content = p_tag.encode_contents().decode('utf-8')
                concatenatedAddressAndOpeningHours += p_html_content + " "
            return parse_addresses_and_hours(concatenatedAddressAndOpeningHours)
    return [[address, opening_hours]]

def checkIfPromoHasMultipleOutlets(deal_soup):
    cards = deal_soup.select('div.details')
    for card in cards:
        header = card.select('h5.header-xs.color-blue')
        if header == []:
            continue
        if header[0].text == "Address":
            # Check if there are multiple addresses, if there are multiple <p> tags
            address_div = card.select_one('div.rte')
            if address_div:
                totalPostalCodes = 0
                for p_tag in address_div.select('p'):
                    totalPostalCodes += countPostalCodesFromRawAddress(p_tag.text)
                return totalPostalCodes > 1
    print("Reached default return while checking mulitple outlets")
    return False


def appendPromoGivenData(promos, title, info, addressesAndOpeningHours, image_url, tagsList, cur_deal_vouchers, deal_link):
    
    entries = len(addressesAndOpeningHours)
    
    if entries == 1:
        address, opening_hours = addressesAndOpeningHours[0]
        longlat = getLongLatFromRawAddress(address)
        
        if longlat in longlat_list:
            # modify the longlat arbitrarily abit here
            pass
        else:
            longlat_lost.append(longlat)
            
        promos.append({
            "title": title,
            "info": info,
            "address": address,
            "image": image_url,
            "opening_hours": opening_hours,
            "tags": tagsList,
            "vouchers": cur_deal_vouchers,
            "link": deal_link,
            "longlat": longlat,
            "id": len(promos)
        })
    else:
        for address, opening_hours in addressesAndOpeningHours:
            
            longlat = getLongLatFromRawAddress(address)
            
            
            
            if longlat in longlat_list:
                
                
                
                # modify the longlat arbitrarily abit here
                pass
            else:
                longlat_lost.append(longlat)
            
            promos.append({
                "title": title,
                "info": info,
                "address": strip_things(address),
                "image": image_url,
                "opening_hours": opening_hours,
                "tags": tagsList,
                "vouchers": cur_deal_vouchers,
                "link": deal_link,
                "longlat": longlat,
                "id": len(promos)
            })

def strip_things(string):
        output = string.replace("<p>",' ')
        output = output.replace("</p>",' ')
        output = output.replace("<br/>"," ")
        output = output.replace("<strong>","")
        output = output.replace("</strong>","")
        return output

In [37]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
# from utils.chopeDetails import parseTags
# from utils.addressConverter import getLongLatFromRawAddress

def get_webscrape_data():
    ## selenium used to scroll to the bottom of the page
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    driver.get("https://shop.chope.co/collections/best-sellers")

    SCROLL_PAUSE_TIME = 0.5

    # Get the height of the page
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new height and check if we have reached the end of the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        
        last_height = new_height

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # getting the relevant data
    all_deals = soup.find(id = 'bc-sf-filter-products')
    url = 'https://shop.chope.co'
    promos = []
    longlat_list = []

    def strip_things(string):
        output = string.replace("<p>",'')
        output = output.replace("</p>",'')
        output = output.replace("<br/>",", ")
        return output

    for deal in all_deals:
        title = deal.select('a.color-blue.app-link') # restaurant / name of deal
        links = deal.select('a')
        link = links[0].get('href') # get url to go to the deal page
        
        deal_link = url+link
        deal_page = requests.get(deal_link)
        deal_soup = BeautifulSoup(deal_page.content, 'html.parser')
        infos = deal_soup.select('div.product-desc')
        info = infos[0].select('strong')[0].text #brief info on deal
        
        # Get voucher details
        vouchers_div = deal_soup.select('div.product-variants.hide-mobile li.child.relative')#deal_soup.select('div.product-variants.hide-mobile')
        cur_deal_vouchers = []
        for voucher in vouchers_div:
            # Extracting date
            date = voucher.select_one('div.date.color-blue.body-s.bold-700')
            if date:
                date = date.text.strip()

            # Extracting time
            time_ = voucher.select_one('div.time.body-s.color-darkgrey.mb-5')
            if time_:
                time_ = time_.text.strip()

            # Extracting price (both discounted and original)
            price_discounted = voucher.select_one('div.product-price strong.price.color-orange')
            if price_discounted:
                price_discounted = price_discounted.text.strip()
            
            price_original = voucher.select_one('div.product-price strike.color-darkgrey.body-s')
            if price_original:
                price_original = price_original.text.strip()

            product_savings = voucher.select_one('div.child-right div.product-savings.big')
            if product_savings:
                product_savings = product_savings.text.strip()

            cur_voucher = {"date": date, "time": time_, "price_discounted": price_discounted, "price_original": price_original, "product_savings": product_savings}
            cur_deal_vouchers.append(cur_voucher)

        cards = deal_soup.select('div.details')

        for card in cards:
            header = card.select('h5.header-xs.color-blue')
            
            if header == []:
                continue

            if header[0].text == "Address":
                address = str(card.select('p')[0])
                address = strip_things(address)  # address of restaurant
            
            if header[0].text == "Opening Hours":
                opening_hours_div = card.select_one('div.rte')
                if opening_hours_div:
                    opening_hours = opening_hours_div.text.replace("\n", "")
            
            if header[0].text == "Cuisine":
                cuisine_div = card.select_one('div.rte')
                if cuisine_div:
                    tags_unparsed = cuisine_div.text
                    tagsList = parseTags(tags_unparsed)

        # get image        
        image_tag = deal.select_one('.product-each-tile-image img')
        image_url = None

        longlat = getLongLatFromRawAddress(address)

        if longlat in longlat_list:
            #adjust the longlat, add a manual arbitrary offset
            pass
        else:
            longlat_list.append(longlat)
        
        
        if image_tag:
            image_url = image_tag['src']
        promos.append({
            "title": title[0].text,
            "info": info,
            "address": address,
            "image": image_url,
            "opening_hours": opening_hours,
            "tags": tagsList,
            "vouchers": cur_deal_vouchers,
            "link": deal_link,
            "longlat": longlat
        })
    return promos

In [137]:
x = get_webscrape_data()

Queried long lat for: CONRAD CENTENNIAL SINGAPORE
Queried long lat for: MORIAH ASSEMBLY OF GOD
Queried long lat for: MARTIN NO 38
Queried long lat for: DEMPSEY CLUSTER (TANGLIN VILLAGE)
Queried long lat for: DBS ION ORCHARD
Queried long lat for: DBS NTUC SCOTTS SQUARE
Queried long lat for: NGEE ANN CITY
Queried long lat for: MORIAH ASSEMBLY OF GOD
Queried long lat for: MTOWER
Queried long lat for: DBS BEDOK MALL 1
Queried long lat for: OCBC COMPASS ONE BRANCH
Queried long lat for: OCBC JEM - FAIRPRICE
Queried long lat for: DBS DBS MARINA BAY SANDS BRANCH
Queried long lat for: NEX
Queried long lat for: DBS PLAZA SINGAPURA BRANCH
Queried long lat for: NEW CREATION CHURCH
Queried long lat for: HSBC JEWEL CHANGI AIRPORT
Queried long lat for: DBS ONE KM
Queried long lat for: NEX
Queried long lat for: DBS CENTERPOINT
Queried long lat for: NEW CREATION CHURCH
Queried long lat for: MORIAH ASSEMBLY OF GOD
Queried long lat for: MY FIRST SKOOL
Queried long lat for: DBS CHANGI AIRPORT TERMINAL 3 B

In [170]:
len(x)

248

In [93]:
longlats = list(df.longlat)

In [132]:
x = copy.deepcopy(z)

In [143]:
z = copy.deepcopy(x)

In [144]:
len(z)

248

In [99]:
type(z)

list

In [94]:
len(longlats)

120

In [121]:
a = 1

In [122]:
a in ls

True

In [120]:
ls = [1,2,3,4]

In [166]:
t = list(z[0]['longlat'][0])

In [168]:
t[0] = float(t[0]) + 0.0002

In [169]:
print(t)

[103.85907951341899, '1.29349139796574']


In [167]:
print(t)

['103.858779513419', '1.29349139796574']


In [175]:
len(df)

248

In [296]:
def create_offsets(n):
    offsets = []
    for i in range(0, (n+1)//2):
        offsets.append(i*0.0002)
        if i != 0: 
            offsets.append(-i*0.0002)
    return offsets 

In [298]:
x = create_offsets(5)
print(x)

[0.0, 0.0002, -0.0002, 0.0004, -0.0004]


In [174]:
count = 0
# z is the dictionary
# get number of occurences for each longlat that has more than one entry
df = pd.DataFrame(z) # convert to datafram
df.longlat = df.longlat.apply(lambda x: tuple(x)) #convert longlat lists to tuples

counts = df.longlat.value_counts()


# function to create the list of offsets depending on how many occurrences there are
def create_offsets(n):
    offsets = []
    for i in range(0, (n+1)//2):
        offsets.append(i*0.0002)
        if i != 0: 
            offsets.append(-i*0.0002)
    return offsets 

    
    


for entry in enumerate(z):
    # print(type(entry['longlat']))
    print(entry[0], '     ', entry[1]['longlat'])
    
    
    
    if entry[1]['longlat'] in [[('103.822596222416', '1.26427434061441')], [('blank')]]:
        print(entry[1]['title'])
        s = list(entry[1]['longlat'][0])
        print('old s: ', s)
        s[0] = str(float(s[0]) + )
        print()
        
        count+=1

print(count)
    #     print(entry)
    #     entry['longlat'] = 'changed'
    #     print(entry)
    #     print()


0       [('103.858779513419', '1.29349139796574')]
1       [('103.822596222416', '1.26427434061441')]
Dancing Crab (VivoCity)
old s:  ['103.822596222416', '1.26427434061441']

2       [('103.838064913731', '1.29213320015397')]
3       [('103.81041169174', '1.30329767270241')]
4       [('103.831592101365', '1.30409806225897')]
5       [('103.832984356543', '1.30585582603508')]
6       [('103.834676124723', '1.30242417189411')]
7       [('103.822596222416', '1.26427434061441')]
Paradise Group
old s:  ['103.822596222416', '1.26427434061441']

8       [('103.801264139551', '1.27398639988204')]
9       [('103.92918919889', '1.32475054237912')]
10       [('103.895063884132', '1.3920543053799')]
11       [('103.743262234833', '1.3325523295069')]
12       [('103.858674928653', '1.28385464447515')]
13       [('103.872296143862', '1.35076935257845')]
14       [('103.845067272938', '1.30082071769235')]
15       [('103.788287276561', '1.30695082420353')]
16       [('103.989989170512', '1.360062893

In [200]:
df.shape

(248, 10)

In [206]:
df.loc[(df['longlat'][0] == ('103.822596222416', '1.26427434061441'))]


KeyError: 'False: boolean label can not be used without a boolean index'

In [204]:
df.loc[(df['opening_hours'] == 'Mon-Sun: 6am-10:30pm')]

,title,info,address,image,opening_hours,tags,vouchers,link,longlat,id
0,Oscar's - 20% Off Buffet,Get 20% off Buffet by purchasing this voucher today!,2 Temasek Boulevard Conrad Centennial Hotel Singapore (038982),https://cdn.shopify.com/s/files/1/0770/5745/products/CCSOscar_s_1_1000x_d05b1e85-4735-493b-9917-211dc8d86038_large.jpg?v=1681791373,Mon-Sun: 6am-10:30pm,"[Buffet, International]","[{'date': 'MON-FRI', 'time': '12:00pm-02:30pm', 'price_discounted': '$61.78', 'price_original': '$77.22', 'product_savings': '-20%'}, {'date': 'MON-TUE', 'time': '06:00pm-10:30pm', 'price_discounted': '$80.78', 'price_original': '$100.98', 'product_savings': '-20%'}, {'date': 'WED-SUN', 'time': '06:00pm-10:30pm', 'price_discounted': '$90.30', 'price_original': '$112.88', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$102.64', 'price_original': '$128.30', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$178.68', 'price_original': '$223.34', 'product_savings': '-20%'}]",https://shop.chope.co/collections/best-sellers/products/oscars-20-off-buffet-new,"[(103.858779513419, 1.29349139796574)]",0


In [184]:
df[df.opening_hours == 'Mon-Sun: 6am-10:30pm']


,title,info,address,image,opening_hours,tags,vouchers,link,longlat,id
0,Oscar's - 20% Off Buffet,Get 20% off Buffet by purchasing this voucher today!,2 Temasek Boulevard Conrad Centennial Hotel Singapore (038982),https://cdn.shopify.com/s/files/1/0770/5745/products/CCSOscar_s_1_1000x_d05b1e85-4735-493b-9917-211dc8d86038_large.jpg?v=1681791373,Mon-Sun: 6am-10:30pm,"[Buffet, International]","[{'date': 'MON-FRI', 'time': '12:00pm-02:30pm', 'price_discounted': '$61.78', 'price_original': '$77.22', 'product_savings': '-20%'}, {'date': 'MON-TUE', 'time': '06:00pm-10:30pm', 'price_discounted': '$80.78', 'price_original': '$100.98', 'product_savings': '-20%'}, {'date': 'WED-SUN', 'time': '06:00pm-10:30pm', 'price_discounted': '$90.30', 'price_original': '$112.88', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$102.64', 'price_original': '$128.30', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$178.68', 'price_original': '$223.34', 'product_savings': '-20%'}]",https://shop.chope.co/collections/best-sellers/products/oscars-20-off-buffet-new,"[(103.858779513419, 1.29349139796574)]",0


In [187]:
len([('103.822596222416', '1.26427434061441')])

1

In [207]:
df[df.tags[0] == ['Buffet', 'International']]

KeyError: True

In [197]:
df[df.longlat == [('103.822596222416', '1.26427434061441')]]

ValueError: ('Lengths must match to compare', (248,), (1, 2))

In [307]:
counts = df.longlat.value_counts()
c2 = counts[counts.values>1]
len(c2)

36

In [308]:
c2

longlat
((103.822596222416, 1.26427434061441),)    9
((103.989989170512, 1.36006289399606),)    7
((103.985169645053, 1.35437074082484),)    6
((103.857705211747, 1.2948461489492),)     5
((103.788287276561, 1.30695082420353),)    4
((103.872296143862, 1.35076935257845),)    4
((103.857007335679, 1.29145526802576),)    4
((103.849984790048, 1.29124400604241),)    4
((103.839434733899, 1.30083890839245),)    4
((103.835978860567, 1.30400640416679),)    3
((103.742586014944, 1.33402529481694),)    3
((103.845067272938, 1.30082071769235),)    3
((103.855411173758, 1.29911287475873),)    3
((103.743262234833, 1.3325523295069),)     3
((103.895063884132, 1.3920543053799),)     3
((103.92918919889, 1.32475054237912),)     3
((103.834676124723, 1.30242417189411),)    3
((103.837286475638, 1.30021292832854),)    2
((103.847617124864, 1.27983487395632),)    2
((103.831980208498, 1.30618594739077),)    2
((103.855901267402, 1.28935082986746),)    2
((103.824907858738, 1.30465158301868),)    2
((

In [311]:
s = iter([1,2,3,4])

In [320]:
a = next(s)
print(a)

StopIteration: 

In [315]:
counts.head()

longlat
((103.822596222416, 1.26427434061441),)    9
((103.989989170512, 1.36006289399606),)    7
((103.985169645053, 1.35437074082484),)    6
((103.857705211747, 1.2948461489492),)     5
((103.788287276561, 1.30695082420353),)    4
Name: count, dtype: int64

In [272]:
df.longlat = df.longlat.apply(lambda x: tuple(x))

In [276]:
df.head(1)

,title,info,address,image,opening_hours,tags,vouchers,link,longlat,id
0,Oscar's - 20% Off Buffet,Get 20% off Buffet by purchasing this voucher today!,2 Temasek Boulevard Conrad Centennial Hotel Singapore (038982),https://cdn.shopify.com/s/files/1/0770/5745/products/CCSOscar_s_1_1000x_d05b1e85-4735-493b-9917-211dc8d86038_large.jpg?v=1681791373,Mon-Sun: 6am-10:30pm,"[Buffet, International]","[{'date': 'MON-FRI', 'time': '12:00pm-02:30pm', 'price_discounted': '$61.78', 'price_original': '$77.22', 'product_savings': '-20%'}, {'date': 'MON-TUE', 'time': '06:00pm-10:30pm', 'price_discounted': '$80.78', 'price_original': '$100.98', 'product_savings': '-20%'}, {'date': 'WED-SUN', 'time': '06:00pm-10:30pm', 'price_discounted': '$90.30', 'price_original': '$112.88', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$102.64', 'price_original': '$128.30', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$178.68', 'price_original': '$223.34', 'product_savings': '-20%'}]",https://shop.chope.co/collections/best-sellers/products/oscars-20-off-buffet-new,"((103.858779513419, 1.29349139796574),)",0


In [274]:
df.longlat[0]

(('103.858779513419', '1.29349139796574'),)

In [280]:
len(df[df.longlat == (('103.822596222416', '1.26427434061441'), )])

9

In [282]:
counts = df.longlat.value_counts()

In [232]:
l.shape

AttributeError: 'list' object has no attribute 'shape'

In [224]:
cdf = counts.reset_index()
cdf.head()

,longlat,count
0,"[(103.822596222416, 1.26427434061441)]",9
1,"[(103.989989170512, 1.36006289399606)]",7
2,"[(103.985169645053, 1.35437074082484)]",6
3,"[(103.857705211747, 1.2948461489492)]",5
4,"[(103.788287276561, 1.30695082420353)]",4


In [300]:
cdf

,longlat,count
0,"[(103.822596222416, 1.26427434061441)]",9
1,"[(103.989989170512, 1.36006289399606)]",7
2,"[(103.985169645053, 1.35437074082484)]",6
3,"[(103.857705211747, 1.2948461489492)]",5
4,"[(103.788287276561, 1.30695082420353)]",4
...,...,...
170,"[(103.844519395251, 1.28435127055659)]",1
171,"[(103.856438251556, 1.30747525502799)]",1
172,"[(103.845030482727, 1.27906414553361)]",1
173,"[(103.852228303832, 1.29755506553528)]",1


In [220]:
type(counts.index)

pandas.core.indexes.base.Index

In [179]:
df.head()

,title,info,address,image,opening_hours,tags,vouchers,link,longlat,id
0,Oscar's - 20% Off Buffet,Get 20% off Buffet by purchasing this voucher today!,2 Temasek Boulevard Conrad Centennial Hotel Singapore (038982),https://cdn.shopify.com/s/files/1/0770/5745/products/CCSOscar_s_1_1000x_d05b1e85-4735-493b-9917-211dc8d86038_large.jpg?v=1681791373,Mon-Sun: 6am-10:30pm,"[Buffet, International]","[{'date': 'MON-FRI', 'time': '12:00pm-02:30pm', 'price_discounted': '$61.78', 'price_original': '$77.22', 'product_savings': '-20%'}, {'date': 'MON-TUE', 'time': '06:00pm-10:30pm', 'price_discounted': '$80.78', 'price_original': '$100.98', 'product_savings': '-20%'}, {'date': 'WED-SUN', 'time': '06:00pm-10:30pm', 'price_discounted': '$90.30', 'price_original': '$112.88', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$102.64', 'price_original': '$128.30', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$178.68', 'price_original': '$223.34', 'product_savings': '-20%'}]",https://shop.chope.co/collections/best-sellers/products/oscars-20-off-buffet-new,"[(103.858779513419, 1.29349139796574)]",0
1,Dancing Crab (VivoCity),Get 20% off Dancing Crab (Vivocity) by purchasing this voucher today!,1 Harbourfront Walk #03-10 VivoCity Singapore (098585),https://cdn.shopify.com/s/files/1/0770/5745/products/2_-_-_Chilli_Crab_Bread_Bowl_-_112_694d2527-459a-48a0-ae5d-cd0a6765842a_large.jpg?v=1628390649,"Mon-Sun: 11:30am-3pm (Last order 2:30pm), 5:30-10pm (Last order 9:30pm)","[American, Cajun, Seafood, Western]","[{'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$40.00', 'price_original': '$50.00', 'product_savings': '-20%'}, {'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$80.00', 'price_original': '$100.00', 'product_savings': '-20%'}]",https://shop.chope.co/collections/best-sellers/products/dancing-crab-vivocity,"[(103.822596222416, 1.26427434061441)]",1
2,Bakalaki Greek Taverna,Get up to 20% off Bakalaki Greek Taverna by purchasing this voucher today!,38A Martin Road Singapore (239059),https://cdn.shopify.com/s/files/1/0770/5745/products/BakalakiBeefSouvlaki_1_large.jpg?v=1628386836,Mon-Sun: 11:30am-12am (Last Order: 10pm),"[Greek, Seafood, Western, Mediterranean]","[{'date': 'MON-FRI', 'time': '11:30am-02:30pm', 'price_discounted': '$40.00', 'price_original': '$50.00', 'product_savings': '-20%'}, {'date': 'MON-FRI, SAT-SUN', 'time': '05:30pm-12:00am, all day', 'price_discounted': '$45.00', 'price_original': '$50.00', 'product_savings': '-10%'}, {'date': 'MON-FRI', 'time': '11:30am-02:30pm', 'price_discounted': '$80.00', 'price_original': '$100.00', 'product_savings': '-20%'}, {'date': 'MON-FRI, SAT-SUN', 'time': '05:30pm-12:00am, all day', 'price_discounted': '$90.00', 'price_original': '$100.00', 'product_savings': '-10%'}]",https://shop.chope.co/collections/best-sellers/products/bakalaki-greek-taverna,"[(103.838064913731, 1.29213320015397)]",2
3,Blu Kouzina (Dempsey),Get up to 20% off Blu Kouzina (Dempsey) by purchasing this voucher today!,10 Dempsey Road #01-20 Singapore (247700),https://cdn.shopify.com/s/files/1/0770/5745/products/Blu_Kouzina_1_large.jpeg?v=1628387561,"Mon-Fri: 11:30am-3:30pm (Last Order: 2:30pm), 5:30-10pm (Last Order: 9:30pm (Food)), 9:45pm (Drinks))Sat-Sun: 8:30-10:30am, 11:30am-3:30pm (Last Order: 2:30pm), 5:30-10pm (Last Order: 9:30pm (Food), 9:45pm (Drinks))",[Greek],"[{'date': 'MON-SUN', 'time': '12:00pm-03:30pm', 'price_discounted': '$80.00', 'price_original': '$100.00', 'product_savings': '-20%'}, {'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$90.00', 'price_original': '$100.00', 'product_savings': '-10%'}, {'date': 'MON-SUN', 'time': '12:00pm-03:30pm', 'price_discounted': '$40.00', 'price_original': '$50.00', 'product_savings': '-20%'}, {'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$45.00', 'price_original': '$50.00', 'product_savings': '-10%'}]",https://shop.chope.co/collections

In [225]:
counts.head()

longlat
[(103.822596222416, 1.26427434061441)]    9
[(103.989989170512, 1.36006289399606)]    7
[(103.985169645053, 1.35437074082484)]    6
[(103.857705211747, 1.2948461489492)]     5
[(103.788287276561, 1.30695082420353)]    4
Name: count, dtype: int64

In [110]:
test = x[0]

In [112]:
test['longlat']

[('103.858779513419', '1.29349139796574')]

In [111]:
print(test)

{'title': "Oscar's - 20% Off Buffet", 'info': 'Get 20% off\xa0Buffet by purchasing this voucher today!', 'address': '2 Temasek Boulevard , Conrad Centennial Hotel,  Singapore (038982)', 'image': 'https://cdn.shopify.com/s/files/1/0770/5745/products/CCSOscar_s_1_1000x_d05b1e85-4735-493b-9917-211dc8d86038_large.jpg?v=1681791373', 'opening_hours': 'Mon-Sun: 6am-10:30pm', 'tags': ['Buffet', 'International'], 'vouchers': [{'date': 'MON-FRI', 'time': '12:00pm-02:30pm', 'price_discounted': '$61.78', 'price_original': '$77.22', 'product_savings': '-20%'}, {'date': 'MON-TUE', 'time': '06:00pm-10:30pm', 'price_discounted': '$80.78', 'price_original': '$100.98', 'product_savings': '-20%'}, {'date': 'WED-SUN', 'time': '06:00pm-10:30pm', 'price_discounted': '$90.30', 'price_original': '$112.88', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$102.64', 'price_original': '$128.30', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30p

In [40]:
x[16]

{'title': 'La Tapería',
 'info': 'Get 20% off La Tapería by purchasing this voucher today!',
 'address': '1 Scotts Road, #02-10/11, Shaw Centre, Singapore (228208)',
 'image': 'https://cdn.shopify.com/s/files/1/0770/5745/products/LaTaperia_Food_Drink_1_6da225d7-f017-4d59-b6a4-b6316f761425_large.jpg?v=1680064493',
 'opening_hours': 'Sun-Thu: 12-3pm (Last Order: 2pm), 6:30-10pm (Last Order 9:30pm)Fri-Sat: 12-3pm (Last Order: 2pm), 6-10pm (Last Order 9:30pm)',
 'tags': ['Spanish'],
 'vouchers': [{'date': 'MON-THU',
   'time': 'All day',
   'price_discounted': '$40.00',
   'price_original': '$50.00',
   'product_savings': '-20%'},
  {'date': 'MON-THU',
   'time': 'All day',
   'price_discounted': '$80.00',
   'price_original': '$100.00',
   'product_savings': '-20%'}],
 'link': 'https://shop.chope.co/collections/best-sellers/products/la-taperia',
 'longlat': [('103.831980208498', '1.30618594739077')]}

In [33]:
for item in enumerate(x[:20]):
    print(item)

(0, {'title': "Oscar's - 20% Off Buffet", 'info': 'Get 20% off\xa0Buffet by purchasing this voucher today!', 'address': '2 Temasek Boulevard , Conrad Centennial Hotel,  Singapore (038982)', 'image': 'https://cdn.shopify.com/s/files/1/0770/5745/products/CCSOscar_s_1_1000x_d05b1e85-4735-493b-9917-211dc8d86038_large.jpg?v=1681791373', 'opening_hours': 'Mon-Sun: 6am-10:30pm', 'tags': ['Buffet', 'International'], 'vouchers': [{'date': 'MON-FRI', 'time': '12:00pm-02:30pm', 'price_discounted': '$61.78', 'price_original': '$77.22', 'product_savings': '-20%'}, {'date': 'MON-TUE', 'time': '06:00pm-10:30pm', 'price_discounted': '$80.78', 'price_original': '$100.98', 'product_savings': '-20%'}, {'date': 'WED-SUN', 'time': '06:00pm-10:30pm', 'price_discounted': '$90.30', 'price_original': '$112.88', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$102.64', 'price_original': '$128.30', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03

In [149]:
import pandas as pd
# pd.set_option('display.max_colwidth', 0)
df = pd.DataFrame(x)

In [150]:
print(df.longlat.value_counts())

longlat
[(103.822596222416, 1.26427434061441)]    9
[(103.989989170512, 1.36006289399606)]    7
[(103.985169645053, 1.35437074082484)]    6
[(103.857705211747, 1.2948461489492)]     5
[(103.788287276561, 1.30695082420353)]    4
                                         ..
[(103.844519395251, 1.28435127055659)]    1
[(103.856438251556, 1.30747525502799)]    1
[(103.845030482727, 1.27906414553361)]    1
[(103.852228303832, 1.29755506553528)]    1
[(103.941582813225, 1.31841298037882)]    1
Name: count, Length: 175, dtype: int64


In [47]:
df['longlat']

0      [(103.858779513419, 1.29349139796574)]
1      [(103.822596222416, 1.26427434061441)]
2      [(103.838064913731, 1.29213320015397)]
3      [(103.81041169174, 1.30329767270241)] 
4      [(103.831592101365, 1.30409806225897)]
                        ...                  
115    [(103.743262234833, 1.3325523295069)] 
116    [(103.839434733899, 1.30083890839245)]
117    [(103.831507365654, 1.30580981580562)]
118    [(103.857705211747, 1.2948461489492)] 
119    [(103.845155395387, 1.27672492702287)]
Name: longlat, Length: 120, dtype: object

In [48]:
df.head()

,title,info,address,image,opening_hours,tags,vouchers,link,longlat
0,Oscar's - 20% Off Buffet,Get 20% off Buffet by purchasing this voucher today!,"2 Temasek Boulevard , Conrad Centennial Hotel, Singapore (038982)",https://cdn.shopify.com/s/files/1/0770/5745/products/CCSOscar_s_1_1000x_d05b1e85-4735-493b-9917-211dc8d86038_large.jpg?v=1681791373,Mon-Sun: 6am-10:30pm,"[Buffet, International]","[{'date': 'MON-FRI', 'time': '12:00pm-02:30pm', 'price_discounted': '$61.78', 'price_original': '$77.22', 'product_savings': '-20%'}, {'date': 'MON-TUE', 'time': '06:00pm-10:30pm', 'price_discounted': '$80.78', 'price_original': '$100.98', 'product_savings': '-20%'}, {'date': 'WED-SUN', 'time': '06:00pm-10:30pm', 'price_discounted': '$90.30', 'price_original': '$112.88', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$102.64', 'price_original': '$128.30', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$178.68', 'price_original': '$223.34', 'product_savings': '-20%'}]",https://shop.chope.co/collections/best-sellers/products/oscars-20-off-buffet-new,"[(103.858779513419, 1.29349139796574)]"
1,Dancing Crab (VivoCity),Get 20% off Dancing Crab (Vivocity) by purchasing this voucher today!,"1 Harbourfront Walk, #03-10, VivoCity, Singapore (098585)",https://cdn.shopify.com/s/files/1/0770/5745/products/2_-_-_Chilli_Crab_Bread_Bowl_-_112_694d2527-459a-48a0-ae5d-cd0a6765842a_large.jpg?v=1628390649,"Mon-Sun: 11:30am-3pm (Last order 2:30pm), 5:30-10pm (Last order 9:30pm)","[American, Cajun, Seafood, Western]","[{'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$40.00', 'price_original': '$50.00', 'product_savings': '-20%'}, {'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$80.00', 'price_original': '$100.00', 'product_savings': '-20%'}]",https://shop.chope.co/collections/best-sellers/products/dancing-crab-vivocity,"[(103.822596222416, 1.26427434061441)]"
2,Bakalaki Greek Taverna,Get up to 20% off Bakalaki Greek Taverna by purchasing this voucher today!,"38A Martin Road, Singapore (239059)",https://cdn.shopify.com/s/files/1/0770/5745/products/BakalakiBeefSouvlaki_1_large.jpg?v=1628386836,Mon-Sun: 11:30am-12am (Last Order: 10pm),"[Greek, Seafood, Western, Mediterranean]","[{'date': 'MON-FRI', 'time': '11:30am-02:30pm', 'price_discounted': '$40.00', 'price_original': '$50.00', 'product_savings': '-20%'}, {'date': 'MON-FRI, SAT-SUN', 'time': '05:30pm-12:00am, all day', 'price_discounted': '$45.00', 'price_original': '$50.00', 'product_savings': '-10%'}, {'date': 'MON-FRI', 'time': '11:30am-02:30pm', 'price_discounted': '$80.00', 'price_original': '$100.00', 'product_savings': '-20%'}, {'date': 'MON-FRI, SAT-SUN', 'time': '05:30pm-12:00am, all day', 'price_discounted': '$90.00', 'price_original': '$100.00', 'product_savings': '-10%'}]",https://shop.chope.co/collections/best-sellers/products/bakalaki-greek-taverna,"[(103.838064913731, 1.29213320015397)]"
3,Blu Kouzina (Dempsey),Get up to 20% off Blu Kouzina (Dempsey) by purchasing this voucher today!,"10 Dempsey Road #01-20, Singapore (247700)",https://cdn.shopify.com/s/files/1/0770/5745/products/Blu_Kouzina_1_large.jpeg?v=1628387561,"Mon-Fri: 11:30am-3:30pm (Last Order: 2:30pm), 5:30-10pm (Last Order: 9:30pm (Food)), 9:45pm (Drinks))Sat-Sun: 8:30-10:30am, 11:30am-3:30pm (Last Order: 2:30pm), 5:30-10pm (Last Order: 9:30pm (Food), 9:45pm (Drinks))",[Greek],"[{'date': 'MON-SUN', 'time': '12:00pm-03:30pm', 'price_discounted': '$80.00', 'price_original': '$100.00', 'product_savings': '-20%'}, {'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$90.00', 'price_original': '$100.00', 'product_savings': '-10%'}, {'date': 'MON-SUN', 'time': '12:00pm-03:30pm', 'price_discounted': '$40.00', 'price_original': '$50.00', 'product_savings': '-20%'}, {'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$45.00', 'price_original': '$50.00', 'product_savings': '-10%'}]",https://shop.chope.co/coll

In [77]:
df.iloc[116].longlat == [('103.839434733899', '1.30083890839245')]

True

In [74]:
type(df.longlat[0])

list

In [75]:
len(df)

120

In [87]:
type(df.longlat[:5][0])

list

In [84]:
df[df.longlat == [(103.858779513419, 1.29349139796574)]]

,title,info,address,image,opening_hours,tags,vouchers,link,longlat


In [79]:
df.head(1)

,title,info,address,image,opening_hours,tags,vouchers,link,longlat
0,Oscar's - 20% Off Buffet,Get 20% off Buffet by purchasing this voucher today!,"2 Temasek Boulevard , Conrad Centennial Hotel, Singapore (038982)",https://cdn.shopify.com/s/files/1/0770/5745/products/CCSOscar_s_1_1000x_d05b1e85-4735-493b-9917-211dc8d86038_large.jpg?v=1681791373,Mon-Sun: 6am-10:30pm,"[Buffet, International]","[{'date': 'MON-FRI', 'time': '12:00pm-02:30pm', 'price_discounted': '$61.78', 'price_original': '$77.22', 'product_savings': '-20%'}, {'date': 'MON-TUE', 'time': '06:00pm-10:30pm', 'price_discounted': '$80.78', 'price_original': '$100.98', 'product_savings': '-20%'}, {'date': 'WED-SUN', 'time': '06:00pm-10:30pm', 'price_discounted': '$90.30', 'price_original': '$112.88', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$102.64', 'price_original': '$128.30', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$178.68', 'price_original': '$223.34', 'product_savings': '-20%'}]",https://shop.chope.co/collections/best-sellers/products/oscars-20-off-buffet-new,"[(103.858779513419, 1.29349139796574)]"


In [78]:
df[df['longlat']==[('103.839434733899', '1.30083890839245')]]

ValueError: ('Lengths must match to compare', (120,), (1, 2))

In [24]:
df.head()

,title,info,address,image,opening_hours,tags,vouchers,link,longlat
0,Oscar's - 20% Off Buffet,Get 20% off Buffet by purchasing this voucher today!,"2 Temasek Boulevard , Conrad Centennial Hotel, Singapore (038982)",https://cdn.shopify.com/s/files/1/0770/5745/products/CCSOscar_s_1_1000x_d05b1e85-4735-493b-9917-211dc8d86038_large.jpg?v=1681791373,Mon-Sun: 6am-10:30pm,"[Buffet, International]","[{'date': 'MON-FRI', 'time': '12:00pm-02:30pm', 'price_discounted': '$61.78', 'price_original': '$77.22', 'product_savings': '-20%'}, {'date': 'MON-TUE', 'time': '06:00pm-10:30pm', 'price_discounted': '$80.78', 'price_original': '$100.98', 'product_savings': '-20%'}, {'date': 'WED-SUN', 'time': '06:00pm-10:30pm', 'price_discounted': '$90.30', 'price_original': '$112.88', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$102.64', 'price_original': '$128.30', 'product_savings': '-20%'}, {'date': 'SAT-SUN', 'time': '12:30pm-03:30pm', 'price_discounted': '$178.68', 'price_original': '$223.34', 'product_savings': '-20%'}]",https://shop.chope.co/collections/best-sellers/products/oscars-20-off-buffet-new,"[(103.858779513419, 1.29349139796574)]"
1,Dancing Crab (VivoCity),Get 20% off Dancing Crab (Vivocity) by purchasing this voucher today!,"1 Harbourfront Walk, #03-10, VivoCity, Singapore (098585)",https://cdn.shopify.com/s/files/1/0770/5745/products/2_-_-_Chilli_Crab_Bread_Bowl_-_112_694d2527-459a-48a0-ae5d-cd0a6765842a_large.jpg?v=1628390649,"Mon-Sun: 11:30am-3pm (Last order 2:30pm), 5:30-10pm (Last order 9:30pm)","[American, Cajun, Seafood, Western]","[{'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$40.00', 'price_original': '$50.00', 'product_savings': '-20%'}, {'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$80.00', 'price_original': '$100.00', 'product_savings': '-20%'}]",https://shop.chope.co/collections/best-sellers/products/dancing-crab-vivocity,"[(103.822596222416, 1.26427434061441)]"
2,Bakalaki Greek Taverna,Get up to 20% off Bakalaki Greek Taverna by purchasing this voucher today!,"38A Martin Road, Singapore (239059)",https://cdn.shopify.com/s/files/1/0770/5745/products/BakalakiBeefSouvlaki_1_large.jpg?v=1628386836,Mon-Sun: 11:30am-12am (Last Order: 10pm),"[Greek, Seafood, Western, Mediterranean]","[{'date': 'MON-FRI', 'time': '11:30am-02:30pm', 'price_discounted': '$40.00', 'price_original': '$50.00', 'product_savings': '-20%'}, {'date': 'MON-FRI, SAT-SUN', 'time': '05:30pm-12:00am, all day', 'price_discounted': '$45.00', 'price_original': '$50.00', 'product_savings': '-10%'}, {'date': 'MON-FRI', 'time': '11:30am-02:30pm', 'price_discounted': '$80.00', 'price_original': '$100.00', 'product_savings': '-20%'}, {'date': 'MON-FRI, SAT-SUN', 'time': '05:30pm-12:00am, all day', 'price_discounted': '$90.00', 'price_original': '$100.00', 'product_savings': '-10%'}]",https://shop.chope.co/collections/best-sellers/products/bakalaki-greek-taverna,"[(103.838064913731, 1.29213320015397)]"
3,Blu Kouzina (Dempsey),Get up to 20% off Blu Kouzina (Dempsey) by purchasing this voucher today!,"10 Dempsey Road #01-20, Singapore (247700)",https://cdn.shopify.com/s/files/1/0770/5745/products/Blu_Kouzina_1_large.jpeg?v=1628387561,"Mon-Fri: 11:30am-3:30pm (Last Order: 2:30pm), 5:30-10pm (Last Order: 9:30pm (Food)), 9:45pm (Drinks))Sat-Sun: 8:30-10:30am, 11:30am-3:30pm (Last Order: 2:30pm), 5:30-10pm (Last Order: 9:30pm (Food), 9:45pm (Drinks))",[Greek],"[{'date': 'MON-SUN', 'time': '12:00pm-03:30pm', 'price_discounted': '$80.00', 'price_original': '$100.00', 'product_savings': '-20%'}, {'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$90.00', 'price_original': '$100.00', 'product_savings': '-10%'}, {'date': 'MON-SUN', 'time': '12:00pm-03:30pm', 'price_discounted': '$40.00', 'price_original': '$50.00', 'product_savings': '-20%'}, {'date': 'MON-SUN', 'time': 'All day', 'price_discounted': '$45.00', 'price_original': '$50.00', 'product_savings': '-10%'}]",https://shop.chope.co/coll